In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import importlib

import load_data

In [ ]:
importlib.reload(load_data)

In [ ]:
df = load_data.read_and_process_parquet()

In [ ]:
file_path = "data/transactions_dataset_reduced.parquet"

df = pd.read_parquet(file_path)
df['date_order'] = pd.to_datetime(df['date_order'])
df['date_invoice'] = pd.to_datetime(df['date_invoice'])

In [ ]:
df.info()

In [ ]:
# Group the data by 'date_order' and 'client_id', and calculate the sum of sales for each group
sales_sum_df = df.groupby(['date_order', 'client_id'])['sales_net'].sum().reset_index()

In [ ]:
sales_sum_df.head()

In [ ]:
df_features = pd.DataFrame(columns=["client_id", "1_month_diff", "3_month_diff", "6_month_diff"])

In [ ]:
df_features

In [ ]:
client_ids = sales_sum_df.client_id.unique()
for client_id in client_ids:
    client_data = sales_sum_df[sales_sum_df['client_id'] == client_id]
    # Sort the data by date_order
    client_data = client_data.sort_values('date_order')
    client_data.drop(columns=['client_id'], inplace=True)
    client_data.set_index('date_order', inplace=True)

    monthly_sales_sum = client_data.resample('M').sum()
    monthly_diff = monthly_sales_sum['sales_net'].diff()

    monthly_sales_sum_3 = client_data.resample('3M').sum()
    monthly_diff_3 = monthly_sales_sum_3['sales_net'].diff()

    monthly_sales_sum_6 = client_data.resample('6M').sum()
    monthly_diff_6 = monthly_sales_sum_6['sales_net'].diff()

    # Add a new row to df_features
    df_features.loc[len(df_features)] = [client_id, monthly_diff.iloc[-1], monthly_diff_3.iloc[-1], monthly_diff_6.iloc[-1]]

    print(f"client: {client_id}, diff: {monthly_diff.iloc[-1]}")


In [ ]:
df_features.tail(200)

In [ ]:
df_features.reset_index(inplace=True, drop=True)

In [ ]:
df_features

In [ ]:
client_id = 1863925
client_data = sales_sum_df[sales_sum_df['client_id'] == client_id]
# Sort the data by date_order
client_data = client_data.sort_values('date_order')

In [ ]:
sales_sum_df[sales_sum_df['client_id'] == 1585335]

In [ ]:
client_data.drop(columns=['client_id'], inplace=True)

In [ ]:
trend = client_data.set_index('date_order')['sales_net'].diff().dropna()

In [ ]:
# Step 2: Check for a consistent decreasing trend
is_decreasing = trend.mean() < 0

In [ ]:
trend.mean()

In [ ]:
is_decreasing

In [ ]:
# Plot sales_net over time
plt.figure(figsize=(8, 4))
plt.plot(client_data['date_order'], client_data['sales_net'], linestyle='-')
plt.xlabel('Date')
plt.ylabel('Sales Net')
plt.title(f'Sales Net Over Time for Client {client_id}')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
client_data.info()

In [ ]:
# Step 2: Set date_order as the index
df = client_data.copy()

# Step 2: Set date_order as the index
df.set_index('date_order', inplace=True)

In [ ]:
df.head()

In [ ]:
# Resample the DataFrame to monthly frequency and calculate the sum
monthly_sales_sum = df.resample('M').sum()

In [ ]:
monthly_sales_sum.tail()

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(monthly_sales_sum.index, monthly_sales_sum['sales_net'], marker='o', linestyle='-')
plt.title('Monthly Sales Net over Time')
plt.xlabel('Date')
plt.ylabel('Sales Net')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
monthly_diff = monthly_sales_sum['sales_net'].diff()

In [ ]:
monthly_diff.tail()

In [ ]:
# Plot the difference in monthly sales net
# Plot percentages
monthly_diff.plot(figsize=(6, 4), marker='o', linestyle='-')
plt.title('Difference in Monthly Sales Net')
plt.xlabel('Date')
plt.ylabel('Difference in Sales Net')
plt.grid(True)
plt.show()

In [ ]:
# Step 3: Calculate rolling sum of sales_net over different time periods
rolling_sum_1m = df['sales_net'].rolling('30D').sum()
rolling_sum_3m = df['sales_net'].rolling('90D').sum()
rolling_sum_6m = df['sales_net'].rolling('180D').sum()

In [ ]:
rolling_sum_1m

In [ ]:
# Step 4: Calculate the change over each time period
change_1m = rolling_sum_1m.diff()
change_3m = rolling_sum_3m.diff()
change_6m = rolling_sum_6m.diff()

In [ ]:
change_1m

In [ ]:
change_1m.iloc[-1]

In [ ]:
change_3m.iloc[-1]

In [ ]:
change_6m.iloc[-1]

In [ ]:
change_1m

In [ ]:
# Step 5: Check if the change over the last period is negative
is_decreasing = change_1m.iloc[-1] < 0 or change_3m.iloc[-1] < 0 or change_6m.iloc[-1] < 0

# Step 6: Determine if the client is at risk for churn
if is_decreasing:
    print("Client is at risk for churn")
else:
    print("Client is not at risk for churn")

In [ ]:
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing the data

# Group the data by client_id and calculate the total sales_net for each client
client_sales = df.groupby('client_id')['sales_net'].sum()

# Identify the three clients with the highest total sales_net
top_clients = client_sales.nlargest(3)

# Identify the three clients with the lowest total sales_net
bottom_clients = client_sales.nsmallest(3)

# Plot sales_net over time for each of the top clients
for client_id in top_clients.index:
    client_data = df[df['client_id'] == client_id]
    client_data = client_data.sort_values('date_order')  # Sort by date_order
    plt.figure()  # Create a new figure for each client
    plt.plot(client_data['date_order'], client_data['sales_net'])
    plt.xlabel('Date')
    plt.ylabel('Sales Net')
    plt.title(f'Sales Net Over Time for Client {client_id}')

# Plot sales_net over time for each of the bottom clients
for client_id in bottom_clients.index:
    client_data = df[df['client_id'] == client_id]
    client_data = client_data.sort_values('date_order')  # Sort by date_order
    plt.figure()  # Create a new figure for each client
    plt.plot(client_data['date_order'], client_data['sales_net'])
    plt.xlabel('Date')
    plt.ylabel('Sales Net')
    plt.title(f'Sales Net Over Time for Client {client_id}')

plt.show()